# **Final Project - SQL | Credit Data Analysis**

This project serves as an introductory exercise in SQL language, developed as the final assignment for the EBAC 'SQL for Data Analysis' course.

I'll be using a database that contains information about clients from a bank.

## About the Data

The dataset has the following columns:

- idade: customer's age
- sexo: customer's sex (F or M)
- dependentes: number of customer's dependents
- escolaridade: education level
- estado_civil: marital status
- salario_anual: annual salary
- tipo_cartao: credit card type 
- qtd_produtos: number of products purchased in the last 12 months
- interacoes_12m: number of interactions made in the last 12 months
- meses_inativo_12m: number of months in which the client was inactive
- limite_credito: credit limit
- valor_transacoes_12m: value of transactions in the last 12 months
- qtd_transacoes_12m: number of transactions made in the last 12 months

In order to explore the data, a set of different queries were created using two tools from Amazon Web Service (AWS): S3 and Athena.
First of all, a specific bucket was created on S3 in which the dataset was uploaded. The, using Athena Query Editor, I created the table using the following query:

~~~
CREATE EXTERNAL TABLE IF NOT EXISTS default.credito (
    idade int,
    sexo string,
    dependentes int,
    escolaridade string,
    estado_civil string,
    salario_anual string,
    tipo_cartao string,
    qtd_produtos bigint,
    interacoes_12m int,
    meses_inativo_12m int,
    limite_credito float,
    valor_transacoes_12m float,
    qtd_transacoes_12m int
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
    'serialization.format' = ',',
    'field.delim' = ','
) LOCATION 's3://projeto-final-sql/'
TBLPROPERTIES ('has_encrypted_data'='false'); 
~~~

## Exploring the Data



### - Table Preview:

SELECT * FROM credito LIMIT 10;

![](https://github.com/noronha-isa/SQL-EBAC/blob/main/preview_table.png?raw=true)

### - How many rows are there in the table?

SELECT COUNT(*) FROM credito;

Answer: 2564 rows

### - Which informations are there in the table?

DESCRIBE credito;

![](https://github.com/noronha-isa/SQL-EBAC/blob/main/describe_table.png?raw=true)

### - What are de different educational levels in this dataset?

SELECT DISTINCT escolaridade FROM credito;

Answer: The dataset has 5 different levels, and some invalid data presented as 'na'. The levels are:
1. sem educacao formal (no formal education)
2. ensino medio (high school)
3. mestrado (master degree)
4. graduacao (bachelor degree)
5. doutorado (PhD)

### - How many customers are categorized in each education level?

~~~
SELECT COUNT(*) AS number_customers, escolaridade as education_level
FROM credito GROUP BY escolaridade;
~~~

Answer: In the column 'escolaridade' there are 346 invalid data ('na'); according to the valid data, the majority of customers have a master degree.

number_customers   | education_level
------------------ | ------
367 | sem educacao formal
545 | ensino medio
261 | graduacao
810 | mestrado
235| doutorado



### - How many women and men are there in this dataset?

~~~
SELECT COUNT(*), sexo FROM credito 
GROUP BY sexo;
~~~

Answer: There are more men than women in this set of data:

number  | sexo
------- | -----
1001 | F
1563 | M


### - Do women spend more money than men using this credit card?

~~~
SELECT max(valor_transacoes_12m) AS max_value, avg(valor_transacoes_12m) AS average_value, sexo AS sex
FROM credito
GROUP BY sexo;
~~~

Answer: No! Both the average and the highest transaction are similar between men and women



![](https://github.com/noronha-isa/SQL-EBAC/blob/main/f_or_m.png?raw=true)

### **- What about the annual salary?**

#### Categories:

~~~
SELECT COUNT(*) AS categories, salario_anual AS annual_salary
FROM credito
GROUP BY salario_anual
ORDER BY salario_anual DESC;
~~~

Comment: There are 235 rows of invalid data (na) for this column;

![](https://github.com/noronha-isa/SQL-EBAC/blob/main/salario_anual.png?raw=true)

### Average age of each salary category:

~~~
SELECT avg(idade) AS avg_age, salario_anual AS annual_salary
FROM credito
WHERE salario_anual != 'na'
GROUP BY salario_anual
ORDER BY salario_anual DESC;
~~~

(excluding invalid data)


![](https://github.com/noronha-isa/SQL-EBAC/blob/main/avg_age.png?raw=true)

### Annual salary according to sex:

~~~
SELECT COUNT(*), sexo AS sex, salario_anual AS annual_salary
FROM credito
WHERE salario_anual != 'na'
GROUP BY sexo, salario_anual
ORDER BY sexo;
~~~

**Comment: There is a huge difference between salaries of men and women ->  there are no women earning more than $ 60k!**

![](https://github.com/noronha-isa/SQL-EBAC/blob/main/salary_men_women.png?raw=true)

### Checking the 'invalid data' for men and women:

~~~
SELECT COUNT (*) AS invalid, sexo AS sex
FROM credito
WHERE salario_anual = 'na'
GROUP BY sexo;
~~~


invalid  | sex
------- | -----
211 | F
24 | M

**Comment: Most of the customers who did not disclose their salary were women; it is possible that the 'na' in this column indicates the absence of income** 


### Does the education level reflect on the salary?

I'll compare the customers with no formal education and the customers with a PhD degree;

~~~
SELECT COUNT (*) AS count, escolaridade AS education, salario_anual AS salary
FROM credito
WHERE escolaridade = 'sem educacao formal' OR escolaridade = 'doutorado'
GROUP BY escolaridade, salario_anual
ORDER BY education, salario_anual DESC;
~~~

Comment: In this dataset, the level of education doesn't seem to affect the annual salary of the customers.

![](https://github.com/noronha-isa/SQL-EBAC/blob/main/education_salary.png?raw=true)